In [5]:
import pandas as pd
import numpy as np
import csv

In [6]:
df = pd.read_csv("../scripts/elabs_extracted.csv", quoting=csv.QUOTE_ALL)


In [7]:
df

,word1,word2,word3,word4
0,vuag,ub,vuag,no
1,xaiv,ntsej,xaiv,muag
2,qhov,phem,qhov,zoo
3,kev,neej,kev,tsav
4,tej,nom,tej,tswv
...,...,...,...,...
3248,khu,mob,khu,nkees
3249,cam,mus,cam,los
3250,me,tes,me,taw
3251,dig,lwj,dig,liam


How many are ABAC vs ABCB?

In [8]:
print('ABAC:', len(df[df.word1==df.word3]))
print('ABCB:', len(df[df.word2==df.word4]))

ABAC: 3253
ABCB: 0


## Test the syllable regex

In [9]:
import sys
sys.path.append("..")
from libraries.hmong_rpa.rpa_regex import RPA_SYLLABLE as rpa

In [10]:
all_syllables = df["word1"].tolist() + df["word2"].tolist() + df["word4"].tolist()

Changed: in all three regex grammars, "ua" is repeated. One of them should be "au"

In [11]:
for syl in all_syllables:
    m = rpa.match(syl)
    ons, rhy, ton = m.group("ons"), m.group("rhy"), m.group("ton")
    if not "rhy":
        print("no vowel for", syl)
    if ons+rhy+ton != syl:
        print(f"wrong composition for {syl}: {ons}+{rhy}+{ton}")

wrong composition for nkawd: nk+aw+
wrong composition for ntawd: nt+aw+
wrong composition for tod: t+o+
wrong composition for ped: p+e+
wrong composition for ped: p+e+
wrong composition for tod: t+o+
wrong composition for ped: p+e+
wrong composition for tod: t+o+
wrong composition for ntusag: nt+u+s
wrong composition for ped: p+e+
wrong composition for nrad: nr+a+
wrong composition for nrad: nr+a+
wrong composition for nrad: nr+a+
wrong composition for nrad: nr+a+
wrong composition for ped: p+e+


One word with a weird "ntusag" syllable. Perhaps a mistake in the extraction process or a typo in the corpus?

In [12]:
df[df.word4=='ntusag']

,word1,word2,word3,word4
362,puj,nrauj,puj,ntusag


A few words that have the 'd' tone, which is not found in rpa. I was going to suggest perhaps these are Dananshan Miao tones, but 'v' doesn't exist in Dananshan (according to Wikipedia) but exist in the same word as the 'd' tones, so I don't know..

In [13]:
df[df.word1.str.endswith('d')|df.word2.str.endswith('d')|df.word4.str.endswith('d')]

,word1,word2,word3,word4
558,plhaw,tod,plhaw,ped
982,rau,ped,rau,nrad
1253,yav,ped,yav,nrad
1377,rau,tod,rau,nrad
2357,xam,ped,xam,nrad
2438,yawg,tod,yawg,ped
3122,nkawd,kwv,nkawd,tij
3137,ntawd,noj,ntawd,haus
